# Web Scraping Project
## import

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import bs4
import pandas as pd
import time

## setup the driver

In [2]:
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=Options())
# Open Shopee website
driver.get("https://shopee.co.th")
driver.maximize_window()

In [3]:
#change language to english in shopee
WebDriverWait(driver, 5).until(EC.visibility_of_element_located((By.XPATH,f"/html/body/div[2]/div[1]/div[1]/div/div[3]/div[2]/button")))
links=driver.find_element(By.XPATH,f"/html/body/div[2]/div[1]/div[1]/div/div[3]/div[2]/button")
links.click()

In [4]:
#close the ads
time.sleep(5)
ads = driver.execute_script('return document.querySelector("shopee-banner-popup-stateful").shadowRoot.querySelector("div.shopee-popup__close-btn")')
ads.click()

In [5]:
catergories = driver.find_element(By.XPATH,f'/html/body/div[1]/div/div[2]/div/div/div[3]/div[3]/div[1]/div/div[2]/div/div[1]/ul/li[20]/a')
catergories.click()

In [6]:
driver.execute_script("document.body.style.zoom = '10%'")
time.sleep(6)

In [7]:
data = driver.page_source
soup = bs4.BeautifulSoup(data)

## Scrape Feature

In [8]:
def k_and_m_to_float(txt): 
    if txt[-1] == "k":
        return float(txt[:-1]) * 1000
    elif txt[-1] == "m":
        return float(txt[:-1]) * 1000000
    else:
        return float(txt)

In [9]:
all_product_name = soup.find_all('div', class_="DgXDzJ rolr6k Zvjf4O")  # Find all product names
name_L = [name.text for name in all_product_name]

all_price = soup.find_all('span', class_="k9JZlv")  # Find all prices
price_L = [float(price.text.replace(',', '')) for price in all_price]

all_sold = soup.find_all('div', class_="OwmBnn eumuJJ")  # Find all sold quantities
sold_L = [k_and_m_to_float(sold.text.removesuffix(" sold")) for sold in all_sold]  # Convert k and m to float

revenue = [price * sold for price, sold in zip(price_L, sold_L)]  # Calculate revenue

## Make to dataframe for use

In [10]:
shopee = pd.DataFrame([name_L,price_L,sold_L,revenue]).transpose()
shopee.columns = ['name','price','sold','revenue']
shopee.dropna(inplace=True) 
shopee.to_csv(r'/Users/thanapatpiyamssakul/coding/first_year/midterm/test.csv') #change the path to your directory